In [1]:
import pandas as pd
import os
import re

In [2]:
base_dir = os.getcwd() 

ruta_grados = os.path.join(base_dir, "Scraping", "tabla_grados.csv")
grados=pd.read_csv("tabla_grados.csv")

grados['nombre_grado'] = grados['nombre_grado'].str.split('/')
grados = grados.explode('nombre_grado')
grados['nombre_grado'] = grados['nombre_grado'].str.strip()

ruta_centros = os.path.join(base_dir, "Scraping", "centros_universitarios.csv")
centros=pd.read_csv("centros_universitarios.csv")

centros = centros[centros["tipo"].notna()]


def limpiar_universidad(valor):
    if pd.isna(valor):
        return valor
    texto = str(valor)

    if re.search(r'\S+@\S+', texto):
        texto = re.sub(r'^.*?\S+@\S+\s*', '', texto)
    
    texto = re.sub(r'(\(|\)|C/).*$', '', texto)

    return texto.strip()

centros["universidad"] = centros["universidad"].apply(limpiar_universidad)
centros["universidad"] = centros["universidad"].replace(r'\d+', '', regex=True)
centros["universidad"] = centros["universidad"].replace(r'["\']', '', regex=True)


In [3]:
centros.head(2)

,nombre_grado,universidad,tipo,facultad,direccion,telefonos,webs,emails
0,Antropología Social y Cultural,Universidad Nacional de Educación a Distancia ...,Pública,Facultad de Filosofía,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,Bellas Artes,Universidad Antonio de Nebrija,Privada,Facultad de Comunicación y Artes Campus de Mad...,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com


# ramas.csv

In [4]:
ramas = pd.DataFrame({
    "id": range(1, len(grados["rama"].unique()) + 1),
    "nombre": grados["rama"].dropna().unique()
})

ramas.to_csv("ramas.csv", index=False, sep=";")

In [5]:
ramas

,id,nombre
0,1,Artes y Humanidades
1,2,Ciencias Sociales y Jurídicas
2,3,Ciencias
3,4,Ciencias de la Salud
4,5,Ingeniería y Arquitectura


# areas.csv

In [6]:
areas = grados[["area", "rama"]].dropna().drop_duplicates().reset_index(drop=True)
areas.insert(0, "id", range(1, len(areas) + 1))
areas = areas.merge(ramas[["id", "nombre"]], left_on="rama", right_on="nombre", how="left")
areas = areas.rename(columns={"nombre": "nombre_rama"})
areas = areas.rename(columns={"id_x": "id", "area": "nombre", "id_y": "id_rama"})
areas = areas[["id", "nombre", "id_rama"]]

areas.to_csv("areas.csv", index=False, sep=";")

In [7]:
areas

,id,nombre,id_rama
0,1,Antropología Social y Cultural,1
1,2,Artes,1
2,3,Diseño,1
3,4,Estudios Culturales,1
4,5,Estudios de Asia y África,1
...,...,...,...
70,71,Ingeniería de la Energía,5
71,72,Ingeniería Matemática,5
72,73,Ingeniería de Materiales,5
73,74,Ingeniería en Telecomunicación,5


# universidad.csv

In [8]:
universidades_unicas = centros["universidad"].dropna().unique()


universidad = pd.DataFrame({
    "id": range(1, len(universidades_unicas) + 1),
    "nombre": universidades_unicas
})

info_universidades = centros.drop_duplicates(subset=["universidad"])[
    ["universidad", "tipo"]
]

universidad = universidad.merge(
    info_universidades,
    left_on="nombre",
    right_on="universidad",
    how="left"
).drop(columns=["universidad"])

universidad.to_csv("universidad.csv", index=False, sep=";")

In [9]:
universidad

,id,nombre,tipo
0,1,Universidad Nacional de Educación a Distancia ...,Pública
1,2,Universidad Antonio de Nebrija,Privada
2,3,Universidad Complutense de Madrid,Pública
3,4,Universidad Francisco de Vitoria,Privada
4,5,Universidad Rey Juan Carlos,Pública
5,6,Universidad Alfonso X El Sabio,Privada
6,7,Universidad San PabloCEU,Privada
7,8,Universidad Europea de Madrid,Privada
8,9,Universidad Carlos III de Madrid,Pública
9,10,Universidad Autónoma de Madrid,Pública


# facultad.csv

In [10]:
facultades_unicas = centros["facultad"].dropna().unique()

facultad = pd.DataFrame({
    "id": range(1, len(facultades_unicas) + 1),
    "nombre": facultades_unicas
})

info_facultades = centros.drop_duplicates(subset=["facultad"])[
    ["facultad", "universidad", "direccion", "telefonos", "webs", "emails"]
]

facultad = facultad.merge(
    info_facultades,
    left_on="nombre",
    right_on="facultad",
    how="left"
).drop(columns=["facultad"])

facultad=facultad.merge(universidad[["id", "nombre"]], left_on="universidad", right_on="nombre")

facultad=facultad[["id_x", "nombre_x", "direccion", "telefonos", "webs", "emails", "id_y"]]
facultad = facultad.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_universidad'})
facultad=facultad.drop_duplicates()

facultad=facultad[["id", "nombre", "id_universidad", "direccion", "telefonos", "webs", "emails"]]

facultad.to_csv("facultad.csv", index=False, sep=";")


In [11]:
facultad

,id,nombre,id_universidad,direccion,telefonos,webs,emails
0,1,Facultad de Filosofía,1,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,24,Facultad de Geografía e Historia,1,"Paseo de la Senda del Rey, 7 28040 Madrid",91 398 67 04,www.uned.es,negociado.geohist@adm.uned.es
2,50,Facultad de Ciencias Políticas y Sociología,1,"C/ Del Obispo Trejo, 2 28040 Madrid",91 398 89 58,www.uned.es,alumnos.poli@adm.uned.es | infouned@adm.uned.es
3,73,Facultad de Ciencias Económicas y Empresariales,1,"Paseo Senda del Rey, 11 28040 Madrid",91 398 63 04,www.uned.es,negociado.economicas@adm.uned.es
4,109,Facultad Ciencias Económicas y Empresariales,1,"Paseo Senda del Rey, 11 28040 Madrid",91 398 63 04,www.uned.es,negociado.economicas@adm.uned.es
...,...,...,...,...,...,...,...
218,142,Facultad de Salud,19,"C/ Castillo de Alarcón, 49 Urb. Villafranca de...",91 815 31 31,www.ucjc.edu,info@ucjc.edu
219,55,Facultad de Derecho Campus Moncloa,21,"C/ Julián Romea, 23 28003 Madrid",91 456 63 00 / 91 514 04 00,www.uspceu.com,info.usp@ceu.es
220,119,Facultad de Farmacia Campus de Montepríncipe,21,Urb. Montepríncipe 28925 Alcorcón (Madrid),91 372 47 00,www.uspceu.com,info.usp@ceu.es
221,68,Facultad de Derecho,24,"C/ Alberto Aguilera, 23 28015 Madrid",91 542 28 00,www.comillas.edu,grado@comillas.edu


# grados.csv

In [12]:
grados_unicos = grados["nombre_grado"].dropna()

grados_tabla = pd.DataFrame({
    "id": range(1, len(grados_unicos) + 1),
    "nombre": grados_unicos
})

info_grados = grados.drop_duplicates(subset=["nombre_grado"])[
    ["nombre_grado", "area", "descripcion", "salidas"]
]

grados_tabla = grados_tabla.merge(
    info_grados,
    left_on="nombre",
    right_on="nombre_grado",
    how="left"
).drop(columns=["nombre_grado"])

grados_tabla=grados_tabla.merge(areas[["id", "nombre"]], left_on="area", right_on="nombre")
grados_tabla=grados_tabla[["id_x", "nombre_x", "id_y", "descripcion", "salidas"]]
grados_tabla = grados_tabla.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_area'})
grados_tabla=grados_tabla.drop_duplicates()
grados_tabla["id"] = range(1, len(grados_tabla) + 1)



grados_tabla.to_csv("grados.csv", index=False, sep=";")

In [13]:
grados_tabla

,id,nombre,id_area,descripcion,salidas
0,1,Antropología Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropología Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
2,3,Antropología Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
3,4,Antropología Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
4,5,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
...,...,...,...,...,...
224,225,Ciencia de Datos e Inteligencia Artificial,75,El grado cubre la creciente necesidad de perfi...,Dichos profesionales estarán capacitados para ...
225,226,Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
226,227,Computación e Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
227,228,Ingeniería Robótica Sotware,75,El objetivo de estas titulaciones es formar pr...,La robótica se ha utilizado desde hace años en...


# grados_ofertados.csv

In [14]:
grados_ofertados=centros[["nombre_grado", "facultad"]]


grados_ofertados=grados_ofertados.merge(grados_tabla[["id", "nombre"]], left_on="nombre_grado", right_on="nombre")
grados_ofertados = grados_ofertados.rename(columns={'id': 'id_grado'})

grados_ofertados=grados_ofertados[['nombre_grado',"facultad", "id_grado"]]

grados_ofertados=grados_ofertados.merge(facultad[["id", "nombre"]], left_on="facultad", right_on="nombre")
grados_ofertados = grados_ofertados.rename(columns={'id': 'id_facultad'})

grados_ofertados[grados_ofertados['nombre_grado'].str.lower() == "matemática computacional"]

grados_ofertados=grados_ofertados[["id_grado", "id_facultad"]]
grados_ofertados=grados_ofertados.drop_duplicates()

grados_ofertados.insert(0, "id", range(1, len(grados_ofertados) + 1))

grados_ofertados.to_csv("grados_ofertados.csv", index=False, sep=";")

In [15]:
grados_ofertados

,id,id_grado,id_facultad
0,1,1,1
1,2,2,1
2,3,3,1
3,4,4,1
4,5,34,1
...,...,...,...
647,587,215,220
648,588,221,221
649,589,222,221
650,590,223,222
